In [68]:
import os

import agama
import h5py
import matplotlib.pyplot as plt
import numpy as np
from gc_utils import iteration_name, main_prog_halt, snapshot_name  # type: ignore
from matplotlib.animation import PillowWriter

In [69]:
sim = "m12i"

sim_dir = "/Users/z5114326/Documents/simulations/"
data_dir = "/Users/z5114326/Documents/GitHub/gc_kinematics/data/"

fire_dir = sim_dir + sim + "/" + sim + "_res7100/"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

agama.setUnits(mass=1, length=1, velocity=1)

In [70]:
it = 70
grp_int = 8580896

it_id = iteration_name(it)
source_dat = proc_data[it_id]["source"]

source_mask = (np.array(source_dat["group_id"]) == grp_int) & (np.array(source_dat["analyse_flag"]) == 1)

snap_acc = np.unique(source_dat["snap_acc"][source_mask])[0]

In [71]:
key_lst = []
for key in proc_data[it_id]["snapshots"].keys():
    if int(key[4:]) >= snap_acc:
        key_lst.append(key)

In [72]:
kin_dict = {}

for key in key_lst:
    snap_dict = proc_data[it_id]["snapshots"][key]
    snap_mask = np.array(snap_dict["group_id"]) == grp_int

    snap = int(key[4:])
    pot_file = data_dir + "potentials/" + sim + "/snap_%d/combined_snap_%d.ini" % (snap, snap)
    pot = agama.Potential(pot_file)
    af = agama.ActionFinder(pot, interp=False)

    et = np.array(snap_dict["et"][snap_mask])
    lz = np.array(snap_dict["lz"][snap_mask])

    r_circs = pot.Rcirc(E=et)
    xyz = np.column_stack((r_circs, r_circs * 0, r_circs * 0))
    v_circs = np.sqrt(-r_circs * pot.force(xyz)[:, 0])
    vel = np.column_stack((v_circs * 0, v_circs, v_circs * 0))
    init_conds = np.concatenate((xyz, vel), axis=1)
    lz_circ = af(init_conds)[:, 2]

    E_0 = pot.potential((0, 0, 0))

    lz_norm = lz / np.array(lz_circ)
    et_norm = et / np.abs(E_0)

    kin_dict[key] = {}

    kin_dict[key]["lz"] = lz
    kin_dict[key]["et"] = et

    kin_dict[key]["lz_norm"] = lz_norm
    kin_dict[key]["et_norm"] = et_norm


# pos = np.array([snap_dict["x"][snap_mask], snap_dict["y"][snap_mask], snap_dict["z"][snap_mask]]).T
# vel = np.array([snap_dict["vx"][snap_mask], snap_dict["vy"][snap_mask], snap_dict["vz"][snap_mask]]).T

# init_cond = np.hstack((pos, vel))

In [73]:
# key = "snap600"

metadata = dict(title="IOM_hist_plot", artist="Finn")
writer = PillowWriter(fps=2, metadata=metadata)

fig = plt.figure(figsize=(16, 16))
fig_shape = (1, 1)

gif_loc = data_dir + "gifs/" + sim

if not os.path.exists(gif_loc):
    os.makedirs(gif_loc)

gif_file = gif_loc + "/" + sim + "_test_" + it_id + ".gif"
with writer.saving(fig, gif_file, 100):
    for key in kin_dict:
        ax = plt.subplot2grid(shape=fig_shape, loc=(0, 0))

        ax.scatter(kin_dict[key]["lz_norm"], kin_dict[key]["et_norm"], s=5)
        ax.set_xlim(-1.1, 1.1)
        ax.set_ylim(-1, 0)

        ax.set_xlabel(r"$\epsilon  \equiv L_{z}$ / $L_{\mathrm{circ}}$(E)")
        ax.set_ylabel(r"$e  \equiv E$ / $|E_{0}|$")

        ax.text(
            +0.5,
            -0.05,
            "Iteration: " + str(it) + "\n" + "Group: " + str(grp_int) + "\n" + key,
            ha="left",
            va="top",
        )
        writer.grab_frame()
        fig.clear()

<Figure size 1600x1600 with 0 Axes>

In [74]:
proc_data.close()